# Instacart Market Basket Analysis

In [98]:
% matplotlib inline

import pandas as pd
from pandas import *
import matplotlib
import matplotlib.pyplot as plt
import plotly as py
#import psycopg2
#import textatistic
import seaborn as sbn
import requests
# matploblib.style.use('ggplot')
# py.offline.init_notebook_mode()
import operator 

In [99]:
orders = pd.read_csv('/home/faculty/millbr02/pub/ecomm/order_products__train.csv')
orders.head(n=10)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1
8,36,39612,1,0
9,36,19660,2,1


### Making a dictionary of order ids, order ids being the key, product ids the value

In [100]:
order_ids = {}

for i,row in orders.iterrows():
    if row['order_id'] not in order_ids:
        order_ids[row['order_id']] = []
    order_ids[row['order_id']].append(row['product_id'])

#### santiy check

In [101]:
order_ids[1]

[49302, 11109, 10246, 49683, 43633, 13176, 47209, 22035]

### Creating fake data to make sure I have the idea right

In [102]:
play_orders = {1:[33, 44, 55, 66], 2:[33, 44, 88], 3:[33, 44, 99], 4:[88, 99], 5:[33, 99], 6:[33, 99, 100]}
play_product_ids = [33, 44, 55, 66, 88, 99, 100]

### Testing fake data

In [75]:
play_matrix = {}

for item in play_product_ids:
    play_matrix[item] = {}
    
for basket in play_orders.values():
    for item in basket:
        valDict = play_matrix[item]
        for other in basket:
            if other != item:
                if other not in valDict:
                    valDict[other] = 1
                else:
                    valDict[other] += 1
print(play_matrix)

{33: {66: 1, 99: 3, 100: 1, 55: 1, 88: 1, 44: 3}, 66: {33: 1, 44: 1, 55: 1}, 99: {88: 1, 33: 3, 44: 1, 100: 1}, 100: {33: 1, 99: 1}, 55: {33: 1, 66: 1, 44: 1}, 88: {33: 1, 99: 1, 44: 1}, 44: {88: 1, 33: 3, 66: 1, 99: 1, 55: 1}}


## Building the real matrix

In [76]:
basket_matrix = {}

for item in orders.product_id.tolist():
    basket_matrix[item] = {}

## Populating the real matrix

In [77]:
for basket in order_ids.values():
    for item in basket:
        valDict = basket_matrix[item]
        for other in basket:
            if other != item:
                if other not in valDict:
                    valDict[other] = 1
                else:
                    valDict[other] += 1

In [78]:
print(basket_matrix[19660])

{16384: 1, 8193: 6, 8195: 1, 49157: 1, 31404: 3, 10: 4, 5463: 1, 16398: 1, 32771: 1, 40982: 1, 49175: 3, 25: 2, 42282: 3, 21850: 1, 36869: 1, 32: 1, 24612: 1, 8230: 2, 49191: 5, 41: 1, 41003: 1, 16428: 1, 45: 4, 41006: 2, 41007: 3, 49: 2, 6835: 1, 1374: 1, 24631: 3, 16443: 1, 32828: 1, 28682: 1, 49215: 3, 16454: 1, 32839: 2, 2732: 1, 11839: 1, 8269: 2, 16462: 1, 79: 2, 8274: 1, 49235: 24, 84: 1, 8277: 19, 32857: 2, 32861: 2, 49247: 2, 32864: 10, 41060: 1, 24680: 1, 41065: 8, 117: 1, 24695: 1, 3550: 1, 16505: 1, 41083: 1, 10944: 1, 24706: 1, 42347: 3, 8324: 1, 24711: 1, 16521: 5, 34262: 1, 23: 1, 16525: 2, 143: 1, 32916: 1, 8341: 1, 155: 1, 8349: 1, 160: 3, 21872: 7, 162: 3, 165: 2, 32935: 1, 24744: 1, 41129: 2, 16556: 1, 16558: 2, 16560: 1, 32950: 2, 24759: 1, 16570: 1, 16415: 2, 24764: 1, 41149: 8, 8382: 2, 8385: 1, 196: 26, 41162: 1, 204: 2, 16589: 2, 210: 2, 32981: 2, 18829: 1, 41177: 3, 219: 2, 8412: 1, 24497: 1, 8414: 3, 24799: 7, 10960: 1, 16611: 2, 228: 1, 49383: 10, 8424: 12, 1

## Given a product id, what other product ids would you predict?
Use product dataset.

### Creating a dataframe out of the products csv file

In [94]:
products = pd.read_csv('/home/faculty/millbr02/pub/ecomm/products.csv')
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


### Merging orders and products dataframes

In [106]:
orders_prods = pd.merge(orders, products, on='product_id')
orders_prods.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1,49302,1,1,Bulgarian Yogurt,120,16
1,816049,49302,7,1,Bulgarian Yogurt,120,16
2,1242203,49302,1,1,Bulgarian Yogurt,120,16
3,1383349,49302,11,1,Bulgarian Yogurt,120,16
4,1787378,49302,8,0,Bulgarian Yogurt,120,16


### Creating a dictionary where keys are product ids, values are product name

In [108]:
order_prod = {}

for i,row in orders_prods.iterrows():
    if row['product_id'] not in order_prod:
        order_prod[row['product_id']] = row['product_name']

### Recommending other items to buy given a product id

In [130]:
# function recommend returns the top 3 items that were in the same basket as the given product id
def recommend(prod_id):
    return dict(sorted(basket_matrix[prod_id].items(), key=operator.itemgetter(1), reverse=True)[:3])

# passing recommend a prod id
prod_id_bought = 47977
top3 = recommend(prod_id_bought)

# finding the product name associated with the top 3 product ids
top3_prod_names = []
for item in top3:
    top3_prod_names.append(str(order_prod[item]))

# printing suggested items 
print("Because you bought {:s}, we recommend that you also buy {:s}, {:s}, and {:s}."
              .format(order_prod[prod_id_bought], top3_prod_names[0], top3_prod_names[1], top3_prod_names[2]))


Because you bought Grapefruit Sparkling Water, we recommend that you also buy Banana, Lemon Sparkling Water, and Passionfruit Sparkling Water.
